# Scope of this Tutorial & Other Resources

This tutorial covers only a small subset of Julia that is necessary for completing the Vahana workshop exercises. While we'll introduce certain types and concepts, we won't delve into deeper aspects like Julia's type system or advanced language features.

For a comprehensive understanding of Julia, please refer to:
- [The official Julia documentation](https://docs.julialang.org/en/v1/)
- [A beginner-friendly tutorial with VSCode introduction](https://www.matecdev.com/posts/julia-introduction-vscode.html)
- [An in-depth MOOC: "Introduction to Scientific Programming and Machine Learning with Julia"](https://sylvaticus.github.io/SPMLJ/stable/)



# Types
## Common Primitive Types in Julia

Julia has several built-in primitive types that are commonly used:

### Integers
Int64 is the default on 64-bit systems

In [1]:
x = 42
typeof(x)

Int64

There are also Int8, Int16, Int32, and their unsigned counterparts (UInt8, UInt16, UInt32, UInt64).

### Floating-point numbers
Float64 (double precision) is the default

In [2]:
y = 3.14
typeof(y)

Float64

### Boolean

In [3]:
is_true = true
typeof(is_true)

Bool

### Strings and Interpolation
Strings can be created with double quotes. Julia provides powerful string interpolation using the `$` symbol:


In [4]:
name = "Julia"
"Hello, $name"  

"Hello, Julia"

Expressions use $():


In [5]:
x = 10
y = 20
"The sum of $x and $y is $(x + y)" 

"The sum of 10 and 20 is 30"

### Symbols
Symbols are immutable names prefixed with a colon. They are commonly used as identifiers or for dictionary keys.


In [6]:
sym = :my_symbol
typeof(sym)  

Symbol

### Tuples
Tuples are immutable fixed-length containers. They can be constructed with parentheses:


In [7]:
point = (1.0, 2.0)
typeof(point)  

Tuple{Float64, Float64}

In [8]:
# Accessing elements (1-based indexing)
x = point[1]  # 1.0

1.0

## Composite Types
Composite types are called records, structs, or objects in various languages. A composite type is a collection of named fields, an instance of which can be treated as a single value. Type annotations in struct definitions are optional in Julia.


In [9]:
struct Point
    x::Float64
    y::Float64
end

p = Point(0.2, 0.4)

Point(0.2, 0.4)

By default, structs in Julia are immutable. This means that once an instance is created, its fields cannot be modified.


In [10]:
p.x = 0.3

ErrorException: setfield!: immutable struct of type Point cannot be changed

To create a mutable struct, you need to explicitly use the 'mutable' keyword.

In [11]:
mutable struct MutablePoint
    x::Float64
    y::Float64
end

p = MutablePoint(0.2, 0.4)
p.x = 0.3

0.3

## Parametric Types
Parametric types in Julia allow type definitions to include type parameters, enabling generic programming with type safety and performance optimization.


In [12]:
vec = Vector() 
push!(vec, 1.2)
typeof(vec)

Vector{Any} (alias for Array{Any, 1})

In [13]:
vec = Vector{Float64}() # alternative vec = Float64[]
push!(vec, 1.2)
typeof(vec)

Vector{Float64} (alias for Array{Float64, 1})

In [14]:
struct PPoint{T}
    x::T
    y::T
end
PPoint(1, 2.3)

MethodError: MethodError: no method matching PPoint(::Int64, ::Float64)
The type `PPoint` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  PPoint(::T, !Matched::T) where T
   @ Main ~/vahana-workshop-preperation/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X34sZmlsZQ==.jl:2


## Bits Types
Bits types in Julia are types whose data is represented as a sequence of bits,
without any references to other values. They are efficient for storage and computation.

Examples of bits types include all primitive types and structs containing only bits types
(The About package can be used to investigate any Julia object.) 


In [15]:
using About
about(Point)

Concrete DataType defined in Main, 16B
  Point <: Any

Struct with 2 fields:
• x  Float64
• y  Float64

 ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
                 8B                                 8B                 


When we create an composite type without typed fields, we can see that this type contains references (pointers)

In [16]:
struct UntypedPoint
    x
    y
end

up = UntypedPoint(0, 2)
about(up)

UntypedPoint (<: Any), occupies 16B directly (referencing 32B in total)
 x::Any 8B Ptr? 0
 y::Any 8B Ptr? 2

 ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
                  *                                  *                 

 * = Pointer (8B)




**In Vahana.jl, all agent (and edge) types must be immutable bits types!**

Which implies that types with a flexible size like Vectors or Strings can not be a field of an agent. 


In [17]:
isbitstype(Vector)

false

In [18]:
isbitstype(String)

false

# Functions

Functions in Julia can be defined in several ways. Here are the most common forms:


In [19]:
# Basic function definition
function add(x, y)
    return x + y
end

# Compact one-line form
add_compact(x, y) = x + y

# Anonymous function
x -> x^2


#11 (generic function with 1 method)

Functions automatically return the value of the last expression. The `return` keyword is optional:


In [20]:
function add(x, y)
    x + y
end

add (generic function with 1 method)

Arguments can have default values:


In [21]:
function power(x, n=2)
    x^n
end


power (generic function with 2 methods)

Arguments and return values can have type annotations using `::`


In [22]:
# Function with type annotations
function add(x::Float64, y::Float64)::Float64
    return x + y
end

add (generic function with 2 methods)

However, it's more common in Julia to omit type annotations unless they're needed for dispatch (will be explained soon) or clarity. This is because Julia's type inference is very powerful, and code without explicit types is often more generic and reusable while maintaining performance.


## Keyword arguments

Keyword arguments follow a semicolon in the function definition and are called by their name:


In [23]:
function plot_point(x, y; color="black", size=1)
    # ... plotting code ...
end

# Calling with keyword arguments
plot_point(2, 3; color="red")
plot_point(2, 3; size=2)
plot_point(2, 3; color="blue", size=2)

## Methods / Multiple dispatch

Julia uses multiple dispatch, where a function can have different methods depending on the types of its arguments:


In [24]:
# Different methods for the same function name
struct Rock end
struct Paper end
struct Scissors end

# Define winning rules using multiple dispatch
beats(::Rock, ::Scissors) = true
beats(::Scissors, ::Paper) = true
beats(::Paper, ::Rock) = true
beats(_, _) = false # `_` is conventionally used as name when the argument  won't be used in the function

# Game function
function play(a, b)
    if beats(a, b)
        "First player wins!"
    elseif beats(b, a)
        "Second player wins!"
    else
        "It's a tie!"
    end
end

# Usage
play(Rock(), Scissors())     

"First player wins!"

In [25]:
play(Rock(), Paper()) 

"Second player wins!"

In [26]:
play(Rock(), Rock())  

"It's a tie!"

Type annotations in methods can be mixed - you only need to specify types for arguments that should participate in dispatch.


## Function Names with `!`

In Julia, functions that modify their arguments end with `!` by convention. This is a naming convention, not a language feature:

In [27]:
# Non-mutating function creates a new array
vec = [3, 1, 2]
sort(vec)    # returns [1, 2, 3], original unchanged
vec

3-element Vector{Int64}:
 3
 1
 2

In [28]:
# Mutating function modifies the input array
sort!(vec)  
vec

3-element Vector{Int64}:
 1
 2
 3

## Pipes
The pipe operator `|>` allows you to chain operations, making code more readable by following a left-to-right flow:


In [29]:
[1,2,3] |> sum |> sqrt # this is equivalent to sqrt(sum([1,2,3]))

2.449489742783178

## Macros
Macros are prefixed with `@` and modify code before it runs. Using common macros is straightforward:


In [30]:
x = 5
@assert x > 0 "x must be positiv"


can be written also as:

In [31]:
@assert(x > 0, "x must be positiv")

# Functional programming constructs: `filter`, `map`, `reduce`

These functions come from functional programming paradigms. While they can be rewritten using loops, 
they often lead to more concise and readable code. Since Vahana.jl provides an own implementation 
of the mapreduce patterns, it's worth understanding them:

## Filter
`filter` creates a new collection containing only elements that satisfy a predicate:

In [32]:
numbers = [1, 2, 3, 4, 5, 6]
even_numbers = filter(x -> x % 2 == 0, numbers) 

3-element Vector{Int64}:
 2
 4
 6


## Map
`map` applies a function to each element of a collection:

In [33]:
numbers = [1, 2, 3]
squares = map(x -> x^2, numbers) 

3-element Vector{Int64}:
 1
 4
 9

## Reduce
`reduce` combines elements of a collection using a binary function:


In [34]:
numbers = [1, 2, 3, 4]
reduce(+, numbers)  

10

## MapReduce
`mapreduce` combines map and reduce into a single operation:


In [35]:
numbers = [1, 2, 3, 4]
mapreduce(x -> x^2, +, numbers)  # equivalent to reduce(+, map(x -> x^2, numbers))

30

## Do-Block Syntax for Function Arguments 
The `do` syntax provides a alternative way to write function arguments that are themselves (anonymous) functions. Since the first argument of functional constructs like `map`, `filter`, and `reduce` is always a function, the `do` notation works particularly well with them:

In [36]:
numbers = [1, 2, 3, 4]

# Single argument example
map(numbers) do x
    if x % 2 == 0
        x^2
    else
        x^3
    end
end  

4-element Vector{Int64}:
  1
  4
 27
 16

In [37]:
# Multiple arguments example
map([2,3,4], [1,2,3]) do x, y
    x + y 
end 

3-element Vector{Int64}:
 3
 5
 7

# Control Flow

## Conditional Statements

### if expression

Julia uses `if`, `elseif`, and `else` for conditional execution. The condition must evaluate to a `Bool`. `if-elseif-else` blocks are expressions that return a value - specifically, the value of the last executed expression in the chosen branch:


In [38]:
function sign_description(x)
    if x < 0
        "negative"
    elseif x == 0
        "zero"
    else
        "positive"
    end
end

sign_description(-2)

"negative"

### Ternary Operator
For simple conditional assignments, the ternary operator `?:` provides a compact syntax:


In [39]:
x = 5
result = x < 0 ? "negative" : "positive"  # equivalent to if x < 0 "negative" else "positive" end

"positive"

### Logical Operators
Julia provides standard logical operators:
- `&&` (AND)
- `||` (OR)
- `!` (NOT)

In [40]:
x = 5
y = 10

if x > 0 && y < 20
    println("both conditions are true")
end

both conditions are true


## Loops
Julia's for loops can iterate over any iterable object:


In [41]:
# Iterate over a range 
for i in 1:3
    println(i)
end


1
2
3


In [42]:
# Iterate over a vector
for i in numbers
    println(i)
end

1
2
3
4


Note (again) that Julia uses 1-based indexing for arrays (unlike Python, C, or JavaScript which use 0-based indexing):

In [43]:
print(numbers)


[1, 2, 3, 4]

In [44]:
numbers[1]

1

## Array Comprehensions and Generators
Julia provides a concise syntax for creating arrays based on existing collections:

In [45]:
squares = [x^2 for x in numbers]  

4-element Vector{Int64}:
  1
  4
  9
 16

With condition

In [46]:
even_squares = [x^2 for x in numbers if x % 2 == 0]  


2-element Vector{Int64}:
  4
 16

 # Modules and Imports

Julia uses modules to organize code into separate namespaces. There are two main ways to access code from other modules: `using` and `import`.

## Using
`using` brings all exported names from a module into the current namespace:



In [47]:
using Statistics  # brings mean, std, etc. into scope

mean(numbers)

2.5

## Import
`import` brings in the module name itself, requiring explicit qualification:

In [48]:
Statistics.mean(numbers)

2.5

The main difference is that with `import`, you always need to prefix the function with the module name, making it explicit where the function comes from. This can help avoid naming conflicts.

You can also specify which function to import, in this case it is not necessary to prefix the function with the module name:

In [49]:
import Statistics: mean 

mean(numbers)

2.5

## Common Pattern
A common pattern in Julia packages is to use both:

In [50]:
using Vahana  # for functions you use frequently, e.g. Vahana
import Random     # for functions where you want to be explicit

## Multiple Modules
You can import multiple modules in one line:


In [51]:
using Vahana, DataFrames
import Random, Graphs

Note: It's considered good practice to put all imports at the beginning of your file.

## Exploring Module Contents

The About package provides a good way to explore the contents of modules:

In [52]:
using About, Vahana
about(Vahana)

Module Vahana [e9033725-1633-496a-b29a-3bc5a5543602]
  Version 1.3.0 loaded from ~/.julia/dev/Vahana

Directly depends on 20 packages:
• MPIPreferences (+1)  • Makie (+220)       • NamedTupleTools   
• MPI (+38)            • GraphMakie (+46)   • Logging           
• ColorSchemes (+8)    • Colors (+6)        • Printf (+1)       
• Graphs (+25)         • Preferences (+1)   • PrettyTables (+13)
• LinearAlgebra (+5)   • Dates              • Metis (+31)       
• StatsBase (+29)      • StaticArrays (+4)  • Requires (+3)     
• HDF5 (+38)           • DataFrames (+26) 

Exports 89 names:
• Vahana                       • neighborids                 
• @rootonly                    • neighborids_iter            
• @roottime                    • neighborstates              
• GlobalsDataFrame             • neighborstates_flexible     
• add_agent!                   • neighborstates_flexible_iter
• add_agent_per_process!       • neighborstates_iter         
• add_agents!                  • num_agen

# Functions/Packages Used in the Workshop Exercises

## Random Operations
Random number generation is used frequently in agent-based models:


In [53]:
rand() # Float64 between 0 and 1

0.5230595839288231

In [54]:
rand(numbers) # draw a random element from a collection

2

For sampling without replacement, use sample from StatsBase:

In [55]:
import StatsBase: sample
sample(1:10, 8; replace=false)  # 8 unique numbers between 1 and 10


8-element Vector{Int64}:
  7
  3
  5
  6
  9
  8
  1
 10

## Benchmarking
### using @time

Julia compiles functions on their first call, which can lead to misleading timing results. 

In [ ]:
using Vahana

struct Agent end

@time ModelTypes() |> register_agenttype!(Agent) |> create_model("BenchmarkExample") |> create_simulation() 

  0.633802 seconds (1.21 M allocations: 61.460 MiB, 15.96% gc time, 93.03% compilation time: 2% of which was recompilation)



Model Name: BenchmarkExamle
Simulation Name: BenchmarkExamle
Agent(s):
	 Type Agent with 0 agent(s)
Still in initialization process!.

In [ ]:
@time ModelTypes() |> register_agenttype!(Agent) |> create_model("BenchmarkExample") |> create_simulation() 

  0.063851 seconds (83.89 k allocations: 3.967 MiB, 26.21% compilation time)



Model Name: BenchmarkExamle
Simulation Name: BenchmarkExamle
Agent(s):
	 Type Agent with 0 agent(s)
Still in initialization process!.


### using BenchmarkTools (or Chairmarks)
The BenchmarkTools package provides robust performance measurements, automatically handling warmup 
(compilation) and running multiple trials for statistical analysis:

In [64]:
using BenchmarkTools
@benchmark ModelTypes() |> register_agenttype!(Agent) |> create_model("BenchmarkExample") |> create_simulation()

BenchmarkTools.Trial: 95 samples with 1 evaluation.
 Range (min … max):  49.816 ms … 98.437 ms  ┊ GC (min … max): 0.00% … 14.47%
 Time  (median):     51.651 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   52.579 ms ±  6.597 ms  ┊ GC (mean ± σ):  0.54% ±  2.02%

   ▃█                                                          
  ▄███▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▁
  49.8 ms         Histogram: frequency by time        95.6 ms <

 Memory estimate: 3.58 MiB, allocs estimate: 75808.